# Introduction to Data Science 2025

# Week 4

In this week's exercise, we look at prompting and zero- and few-shot task settings. Below is a text generation example from https://github.com/TurkuNLP/intro-to-nlp/blob/master/text_generation_pipeline_example.ipynb demonstrating how to load a text generation pipeline with a pre-trained model and generate text with a given prompt. Your task is to load a similar pre-trained generative model and assess whether the model succeeds at a set of tasks in zero-shot, one-shot, and two-shot settings.

**Note: Downloading and running the pre-trained model locally may take some time. Alternatively, you can open and run this notebook on [Google Colab](https://colab.research.google.com/), as assumed in the following example.**

## Text generation example

This is a brief example of how to run text generation with a causal language model and `pipeline`.

Install [transformers](https://huggingface.co/docs/transformers/index) python package. This will be used to load the model and tokenizer and to run generation.

In [19]:
!pip install --quiet transformers

Import the `AutoTokenizer`, `AutoModelForCausalLM`, and `pipeline` classes. The first two support loading tokenizers and generative models from the [Hugging Face repository](https://huggingface.co/models), and the last wraps a tokenizer and a model for convenience.

In [20]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

Load a generative model and its tokenizer. You can substitute any other generative model name here (e.g. [other TurkuNLP GPT-3 models](https://huggingface.co/models?sort=downloads&search=turkunlp%2Fgpt3)), but note that Colab may have issues running larger models. 

In [21]:
MODEL_NAME = 'TurkuNLP/gpt3-finnish-large'

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

Instantiate a text generation pipeline using the tokenizer and model.

In [22]:
pipe = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    device=model.device
)

We can now call the pipeline with a text prompt; it will take care of tokenizing, encoding, generation, and decoding:

In [23]:
output = pipe('Terve, miten menee?', max_new_tokens=25)

print(output)

[{'generated_text': 'Terve, miten menee?”\n”Hyvin, kiitos.”\n”Kiva kuulla.”\n”Kuule, minulla on sinulle asiaa.”\n'}]


Just print the text

In [24]:
print(output[0]['generated_text'])

Terve, miten menee?”
”Hyvin, kiitos.”
”Kiva kuulla.”
”Kuule, minulla on sinulle asiaa.”



We can also call the pipeline with any arguments that the model `generate` function supports. For details on text generation using `transformers`, see e.g. [this tutorial](https://huggingface.co/blog/how-to-generate).

Example with sampling and a high `temperature` parameter to generate more chaotic output:

In [26]:
output = pipe(
    'Terve, miten menee?',
    do_sample=True,
    temperature=10.0,
    max_new_tokens=25
)

print(output[0]['generated_text'])

Terve, miten menee? kysyi Heikki yhtäkkiä astuessaan taloon sisään kantaen kahvipöytä-tarvikkeita käsissään sisään tultuaan.
(Ryökäle luuli varmasti hänen tulevan meille istumaan)




## Exercise 1

Your task is to assess whether a generative model succeeds in the following tasks in zero-shot, one-shot, and two-shot settings:

- binary sentiment classification (positive / negative)

- person name recognition

- two-digit addition (e.g. 11 + 22 = 33)

For example, for assessing whether a generative model can name capital cities, we could use the following prompts:

- zero-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- one-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""
- two-shot:
	>"""\
	>Identify the capital cities of countries.
	>
	>Question: What is the capital of Sweden?\
	>Answer: Stockholm
	>
	>Question: What is the capital of Denmark?\
	>Answer: Copenhagen
	>
	>Question: What is the capital of Finland?\
	>Answer:\
	>"""

You can do the tasks either in English or Finnish and use a generative model of your choice from the Hugging Face models repository, for example the following models:

- English: `gpt2-large`
- Finnish: `TurkuNLP/gpt3-finnish-large`

You can either come up with your own instructions for the tasks or use the following:

- English:
	- binary sentiment classification: "Do the following texts express a positive or negative sentiment?"
	- person name recognition: "List the person names occurring in the following texts."
	- two-digit addition: "This is a first grade math exam."
- Finnish:
	- binary sentiment classification: "Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?"
	- person name recognition: "Listaa seuraavissa teksteissä mainitut henkilönnimet."
	- two-digit addition: "Tämä on ensimmäisen luokan matematiikan koe."

Come up with at least two test cases for each of the three tasks, and come up with your own one- and two-shot examples.

In [ ]:
output_zero_shot = pipe('Ilmaiseeko seuraava teksti positiivista vai negatiivista tunnetta?\nTeksti: Olen raivoissani\nVastaus:', max_new_tokens=2)
print(output_zero_shot[0]['generated_text'])

output_one_shot = pipe('Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?\nTeksti: Toivottomuuden tunne on poissa\nVastaus: Positiivisia\nTeksti: Olen raivoissani\nVastaus:', max_new_tokens=2)
print(output_one_shot[0]['generated_text'])

output_two_shot = pipe('Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?\nTeksti: Olen raivoissani\nVastaus: Negatiivisia\nTeksti:Toivottomuuden tunne on poissa\nVastaus: Positiivisia\nTeksti: Olen raivoissani\nVastaus:', max_new_tokens=2)
print(output_two_shot[0]['generated_text'])

output_zero_shot_names = pipe('Listaa seuraavissa teksteissä mainitut henkilönnimet.\nTeksti: Maija ja Juho menivät elokuviin.\nVastaus:', max_new_tokens=2)
print(output_zero_shot_names[0]['generated_text'])

output_one_shot_names = pipe('Listaa seuraavissa teksteissä mainitut henkilönnimet.\nTeksti: Pekka ja Maija menivät kauppaan.\nVastaus: Pekka Maija\nTeksti: Juho ja Liisa ovat ystäviä.\nVastaus:', max_new_tokens=2)
print(output_one_shot_names[0]['generated_text'])

output_two_shot_names = pipe('Listaa seuraavissa teksteissä mainitut henkilönnimet.\nTeksti: Pekka ja Maija menivät kauppaan.\nVastaus: Pekka Maija\nTeksti: Juha on opettaja.\nVastaus: Juha\nTeksti: Sanna ja Antti ovat naimisissa.\nVastaus:', max_new_tokens=2)
print(output_two_shot_names[0]['generated_text'])

output_zero_shot_addition = pipe('Tämä on ensimmäisen luokan matematiikan koe.\nLaske seuraava summa:\n23 + 45 =\nVastaus:', max_new_tokens=2)
print(output_zero_shot_addition[0]['generated_text'])

output_one_shot_addition = pipe('Tämä on ensimmäisen luokan matematiikan koe.\nLaske seuraava summa:\n12 + 34 =\nVastaus: 46\nLaske seuraava summa:\n23 + 45 =\nVastaus:', max_new_tokens=2)
print(output_one_shot_addition[0]['generated_text'])

output_two_shot_addition = pipe('Tämä on ensimmäisen luokan matematiikan koe.\nLaske seuraava summa:\n12 + 34 =\nVastaus: 46\nLaske seuraava summa:\n50 + 10 =\nVastaus: 60\nLaske seuraava summa:\n23 + 45 =\nVastaus:', max_new_tokens=2)
print(output_two_shot_addition[0]['generated_text'])

Ilmaiseeko seuraava teksti positiivista vai negatiivista tunnetta?
Teksti: Olen raivoissani
Vastaus: Voi kuulostaa

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Toivottomuuden tunne on poissa
Vastaus: Positiivisia
Teksti: Olen raivoissani
Vastaus: Positiivisia

Ilmaisevatko seuraavat tekstit positiivista vai negatiivista tunnetta?
Teksti: Olen raivoissani
Vastaus: Negatiivisia
Teksti:Toivottomuuden tunne on poissa
Vastaus: Positiivisia
Teksti: Olen raivoissani
Vastaus: Positiivisia


Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Maija ja Juho menivät elokuviin.
Vastaus:
Maija

Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Pekka ja Maija menivät kauppaan.
Vastaus: Pekka Maija
Teksti: Juho ja Liisa ovat ystäviä.
Vastaus: Juho Liisa

Listaa seuraavissa teksteissä mainitut henkilönnimet.
Teksti: Pekka ja Maija menivät kauppaan.
Vastaus: Pekka Maija
Teksti: Juha on opettaja.
Vastaus: Juha
Teksti: Sanna ja Antti ovat naimisissa.
Vastaus: Sanna ja

Tämä on ensimmäisen luokan matematiikan koe.
Laske seuraava summa:
23 + 45 =
Vastaus:
Oikea

Laske seuraava summa:
12 + 34 =
Vastaus: 46
Laske seuraava summa:
23 + 45 =
Vastaus: 40

Laske seuraava summa:
12 + 34 =
Vastaus: 46
Laske seuraava summa:
50 + 10 =
Vastaus: 60
Laske seuraava summa:
23 + 45 =
Vastaus: 65

The feeling task required two tokens instead of two but that might be related to the expected words being a tad longer than usual. The model failed the feeling task generally speaking.

Detecting names from texts worked fine with one-shot but failed in different ways in zero-shot and two-shot.

Math task failed in zero-shot probably due to trying to form a sentence rather than a number. The other few-shots got the number format right but failed in calculating it. No hard feelings since it was mainly a finnish-specific GPT.

**Submit this exercise by submitting your code and your answers to the above questions as comments on the MOOC platform. You can return this Jupyter notebook (.ipynb) or .py, .R, etc depending on your programming preferences.**